In [ ]:
# Eliminacion de la base de datos
# partidos_collection.delete_many({})

In [9]:
from datetime import datetime
from pymongo import MongoClient
from tabulate import tabulate

In [10]:
# Conexión a la base de datos
client = MongoClient('mongodb://localhost:27017')
db = client['campeonato']
partidos_collection = db['partidos']

In [22]:
# Función para ingresar los resultados de los partidos
def ingresar_resultados(fecha, resultados):
    for partido, resultado in resultados.items():
        partido_data = {
            'fecha': fecha,
            'partido': partido,
            'resultado': resultado
        }
        partidos_collection.insert_one(partido_data)

# Función para obtener la tabla de posiciones general
def obtener_tabla_posiciones():
    tabla_posiciones = {}
    for partido in partidos_collection.find():
        resultado = partido['resultado']
        equipo_local, goles_local = resultado['local'], resultado['goles_local']
        equipo_visitante, goles_visitante = resultado['visitante'], resultado['goles_visitante']

        if equipo_local not in tabla_posiciones:
            tabla_posiciones[equipo_local] = {'PJ': 0, 'PG': 0, 'PE': 0, 'PP': 0, 'GF': 0, 'GC': 0, 'GD':0, 'PTS': 0}
        if equipo_visitante not in tabla_posiciones:
            tabla_posiciones[equipo_visitante] = {'PJ': 0, 'PG': 0, 'PE': 0, 'PP': 0, 'GF': 0, 'GC': 0, 'GD':0, 'PTS': 0}

        tabla_posiciones[equipo_local]['PJ'] += 1
        tabla_posiciones[equipo_visitante]['PJ'] += 1

        tabla_posiciones[equipo_local]['GF'] += goles_local
        tabla_posiciones[equipo_local]['GC'] += goles_visitante
        tabla_posiciones[equipo_visitante]['GF'] += goles_visitante
        tabla_posiciones[equipo_visitante]['GC'] += goles_local
        tabla_posiciones[equipo_local]['GD'] = tabla_posiciones[equipo_local]['GF'] - tabla_posiciones[equipo_local]['GC']
        tabla_posiciones[equipo_visitante]['GD'] = tabla_posiciones[equipo_visitante]['GF'] - tabla_posiciones[equipo_visitante]['GC']

        if goles_local > goles_visitante:
            tabla_posiciones[equipo_local]['PG'] += 1
            tabla_posiciones[equipo_visitante]['PP'] += 1
        elif goles_local < goles_visitante:
            tabla_posiciones[equipo_local]['PP'] += 1
            tabla_posiciones[equipo_visitante]['PG'] += 1
        else:
            tabla_posiciones[equipo_local]['PE'] += 1
            tabla_posiciones[equipo_visitante]['PE'] += 1

    for equipo, datos in tabla_posiciones.items():
        datos['PTS'] = 3 * datos['PG'] + 1 * datos['PE']

    return tabla_posiciones

In [23]:
# Ingresar resultados de los partidos
resultados_fecha1 = {
    'Partido1': {'local': 'EquipoA', 'goles_local': 2, 'visitante': 'EquipoB', 'goles_visitante': 1},
    'Partido2': {'local': 'EquipoC', 'goles_local': 0, 'visitante': 'EquipoD', 'goles_visitante': 0},
    'Partido3': {'local': 'EquipoE', 'goles_local': 1, 'visitante': 'EquipoF', 'goles_visitante': 3}
}

resultados_fecha2 = {
    'Partido1': {'local': 'EquipoA', 'goles_local': 3, 'visitante': 'EquipoC', 'goles_visitante': 1},
    'Partido2': {'local': 'EquipoB', 'goles_local': 2, 'visitante': 'EquipoE', 'goles_visitante': 2},
    'Partido3': {'local': 'EquipoD', 'goles_local': 1, 'visitante': 'EquipoF', 'goles_visitante': 2}
}

ingresar_resultados(datetime(2023, 7, 1), resultados_fecha1)
ingresar_resultados(datetime(2023, 7, 8), resultados_fecha2)

In [24]:
# Obtener la tabla de posiciones general
tabla_posiciones = obtener_tabla_posiciones()

# Ordenar la tabla de posiciones por puntos, gol diferencia y goles a favor
tabla_ordenada = sorted(tabla_posiciones.items(), key=lambda x: (x[1]['PTS'], x[1]['GD'], x[1]['GF']), reverse=True)

# Imprimir la tabla de posiciones
tabla = []
for equipo, datos in tabla_ordenada:
    fila = [equipo, datos['PJ'], datos['PG'], datos['PE'], datos['PP'], datos['GF'], datos['GC'], datos['GD'], datos['PTS']]
    tabla.append(fila)

encabezados = ["Equipo", "PJ", "PG", "PE", "PP", "GF", "GC", "GD", "PTS"]

print("Tabla de Posiciones:")
print(tabulate(tabla, headers=encabezados, tablefmt="fancy_grid"))

Tabla de Posiciones:
╒══════════╤══════╤══════╤══════╤══════╤══════╤══════╤══════╤═══════╕
│ Equipo   │   PJ │   PG │   PE │   PP │   GF │   GC │   GD │   PTS │
╞══════════╪══════╪══════╪══════╪══════╪══════╪══════╪══════╪═══════╡
│ EquipoA  │    4 │    4 │    0 │    0 │   10 │    4 │    6 │    12 │
├──────────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ EquipoF  │    4 │    4 │    0 │    0 │   10 │    4 │    6 │    12 │
├──────────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ EquipoB  │    4 │    0 │    2 │    2 │    6 │    8 │   -2 │     2 │
├──────────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ EquipoD  │    4 │    0 │    2 │    2 │    2 │    4 │   -2 │     2 │
├──────────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ EquipoE  │    4 │    0 │    2 │    2 │    6 │   10 │   -4 │     2 │
├──────────┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼───────┤
│ EquipoC  │    4 │    0 │    2 │    2 │    2 │    6 │   -4 │     2 │